## LSDB Distant Recurrence Notreatment _5Year_NoDays_Digital

In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV #网格搜索
from sklearn.model_selection import train_test_split #将数据集分开成训练集和测试集
from sklearn.model_selection import StratifiedShuffleSplit #分层采样
import numpy as np
from sklearn.model_selection import GridSearchCV
import math

In [4]:
Distant_Recurrence_5Year_data = pd.read_table('LSDBDistantRecurrenceNotreatment_5Year_NoDays_Digital.txt',delim_whitespace=True)

In [5]:
Distant_Recurrence_5Year_data.head()

,race,ethnicity,smoking,alcohol_useage,family_history,age_at_diagnosis,menopause_status,side,TNEG,ER,...,size,grade,invasive,histology2,invasive_tumor_Location,DCIS_level,re_excision,surgical_margins,MRIs_60_surgery,distant_recurrence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,0,0,0,0,...,1,1,0,0,0,1,0,0,0,0
2,0,0,0,0,2,2,1,0,0,0,...,0,1,0,0,1,2,0,0,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,2,1,0,0,1,0
4,0,0,0,0,2,1,0,0,0,0,...,0,2,0,0,2,0,0,0,1,0


In [6]:
type(Distant_Recurrence_5Year_data)

pandas.core.frame.DataFrame

In [7]:
Distant_Recurrence_5Year_data.dtypes

race                       int64
ethnicity                  int64
smoking                    int64
alcohol_useage             int64
family_history             int64
age_at_diagnosis           int64
menopause_status           int64
side                       int64
TNEG                       int64
ER                         int64
ER_percent                 int64
PR                         int64
PR_percent                 int64
P53                        int64
HER2                       int64
t_tnm_stage                int64
n_tnm_stage                int64
stage                      int64
lymph_node_removed         int64
lymph_node_positive        int64
lymph_node_status          int64
Histology                  int64
size                       int64
grade                      int64
invasive                   int64
histology2                 int64
invasive_tumor_Location    int64
DCIS_level                 int64
re_excision                int64
surgical_margins           int64
MRIs_60_su

In [8]:
Distant_Recurrence_5Year_data.shape[1]

32

In [9]:
Distant_Recurrence_5Year_data['distant_recurrence'].value_counts()

0    3752
1     437
Name: distant_recurrence, dtype: int64

In [10]:
437/(3752+437)

0.10432084029601336

In [11]:
#五年复发概率大约是10%，我们的测试集需要按照10% positive 90%negative来划分吗

In [12]:
Distant_Recurrence_5Year_data[Distant_Recurrence_5Year_data['distant_recurrence']==1][:10]

,race,ethnicity,smoking,alcohol_useage,family_history,age_at_diagnosis,menopause_status,side,TNEG,ER,...,size,grade,invasive,histology2,invasive_tumor_Location,DCIS_level,re_excision,surgical_margins,MRIs_60_surgery,distant_recurrence
18,0,0,0,0,0,2,1,1,0,0,...,2,2,0,0,2,5,1,0,0,1
21,1,0,1,1,1,0,0,1,0,0,...,0,0,0,2,3,3,0,0,0,1
22,1,0,0,0,4,1,0,0,0,0,...,0,0,0,0,2,1,0,0,1,1
25,0,0,1,1,2,2,1,1,0,1,...,1,0,0,0,0,0,0,0,1,1
26,0,0,1,0,1,2,1,1,0,1,...,0,2,0,0,2,1,0,0,1,1
42,0,0,1,1,2,1,0,0,1,1,...,2,2,0,0,2,5,0,0,0,1
57,0,0,1,2,4,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
59,0,0,1,2,2,1,0,0,0,0,...,1,0,0,0,2,0,0,0,0,1
71,0,0,2,2,1,1,0,1,0,0,...,1,1,0,2,3,1,1,0,0,1
80,1,0,1,1,1,1,0,1,0,0,...,2,2,0,0,0,2,0,0,0,1


### Separate Train and Test 80%,20%

In [13]:
x = Distant_Recurrence_5Year_data.iloc[:,0:Distant_Recurrence_5Year_data.shape[1]-1]

In [14]:
x.shape

(4189, 31)

In [15]:
y = Distant_Recurrence_5Year_data.iloc[:,Distant_Recurrence_5Year_data.shape[1]-1:Distant_Recurrence_5Year_data.shape[1]]

In [16]:
y.shape

(4189, 1)

In [17]:
#数据集划分(采用分层划分保证稳定性)
seed = 7
test_size = 0.2 #20% positive 80% negative

In [18]:
split_machine = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)

In [19]:
for train_index,test_index in split_machine.split(x,y):
    train_set = Distant_Recurrence_5Year_data.loc[train_index]
    test_set = Distant_Recurrence_5Year_data.loc[test_index]

In [20]:
print(len(train_set),len(test_set))

3351 838


In [30]:
838/(3351+838)

0.20004774409166864

In [22]:
test_set.head()

,race,ethnicity,smoking,alcohol_useage,family_history,age_at_diagnosis,menopause_status,side,TNEG,ER,...,size,grade,invasive,histology2,invasive_tumor_Location,DCIS_level,re_excision,surgical_margins,MRIs_60_surgery,distant_recurrence
3822,1,0,2,0,3,1,0,0,0,1,...,2,2,0,0,2,0,0,0,1,0
2945,0,0,1,0,2,2,1,0,0,0,...,1,0,0,2,3,0,0,0,0,0
3193,0,0,0,0,2,1,0,1,0,0,...,2,1,0,0,2,3,0,0,1,1
506,2,0,1,1,0,2,1,0,0,0,...,1,0,1,1,2,0,0,0,1,0
1407,0,0,1,0,0,1,0,0,0,0,...,1,0,1,1,2,3,1,3,0,0


In [28]:
# 检查测试集结果是否符合分类预期
test_set["distant_recurrence"].value_counts() / len(test_set)

0    0.896181
1    0.103819
Name: distant_recurrence, dtype: float64

In [29]:
# 检查训练集结果是否符合分类预期
train_set["distant_recurrence"].value_counts() / len(train_set)

0    0.895554
1    0.104446
Name: distant_recurrence, dtype: float64

In [51]:
## 5-fold cross validation SVM
X_train = train_set.iloc[:,0:train_set.shape[1]-1]

In [52]:
X_test = test_set.iloc[:,0:train_set.shape[1]-1]

In [53]:
y_train = train_set.iloc[:,Distant_Recurrence_5Year_data.shape[1]-1:Distant_Recurrence_5Year_data.shape[1]]

In [54]:
y_test = test_set.iloc[:,Distant_Recurrence_5Year_data.shape[1]-1:Distant_Recurrence_5Year_data.shape[1]]

In [39]:
y_train = test_set.iloc[:,0:train_set.shape[1]-1]

In [41]:
y_test = test_set.iloc[:,Distant_Recurrence_5Year_data.shape[1]-1:Distant_Recurrence_5Year_data.shape[1]]

In [55]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel = 'linear',random_state = 0)
svm_classifier.fit(X_train,y_train)

C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [56]:
y_pred = svm_classifier.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [58]:
cm

array([[751,   0],
       [ 87,   0]], dtype=int64)

In [59]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test,y_pred)

In [60]:
accuracy

0.89618138424821

In [72]:
## GridSearch
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[math.pow(2,-5),math.pow(2,-1),math.pow(2,3),math.pow(2,7),math.pow(2,11)],'kernel':['linear']},
              {'C':[math.pow(2,-5),math.pow(2,-1),math.pow(2,3),math.pow(2,7),math.pow(2,11)],'kernel':['rbf'],'gamma':[-11,-7,-3,1,5]}]
grid_search_svm = GridSearchCV(estimator = svm_classifier,
                              param_grid = parameters,
                              scoring = 'roc_auc',
                              cv = 5,
                              )
grid_search_svm = grid_search_svm.fit(X_train,np.ravel(y_train))

C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: gamma < 0

  FitFailedWarning)
C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: gamma < 0

  FitFailedWarning)
C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: gamma < 0

  FitFailedWarning)
C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Val

C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: gamma < 0

  FitFailedWarning)
C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: gamma < 0

  FitFailedWarning)
C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: gamma < 0

  FitFailedWarning)
C:\Users\xuchu\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Val

In [73]:
grid_search_svm.best_score_

0.6649074082877744

In [76]:
grid_search_svm.best_params_

{'C': 0.5, 'kernel': 'linear'}

In [79]:
grid_search_svm.cv_results_

{'mean_fit_time': array([2.01051998e-01, 2.28143263e-01, 1.16290698e+00, 1.11588257e+01,
        1.48014627e+02, 7.12928772e-03, 6.99391365e-03, 6.62093163e-03,
        9.62711239e-01, 9.63923168e-01, 6.84170723e-03, 7.58686066e-03,
        7.59696960e-03, 1.03204546e+00, 1.04009633e+00, 6.18124008e-03,
        5.98893166e-03, 6.25467300e-03, 1.16681471e+00, 1.28024468e+00,
        6.67486191e-03, 5.95588684e-03, 5.86690903e-03, 1.14136720e+00,
        1.25552406e+00, 6.00209236e-03, 6.30660057e-03, 6.77223206e-03,
        1.14478669e+00, 1.25556922e+00]),
 'std_fit_time': array([2.51907827e-02, 7.36439199e-03, 8.64865316e-02, 3.00303921e-01,
        3.09181720e+00, 1.18384868e-03, 1.88411630e-05, 8.03361744e-04,
        2.09929741e-02, 1.85356564e-02, 9.22978971e-04, 1.17848304e-03,
        7.99782236e-04, 2.43882365e-02, 3.41105196e-02, 7.18808221e-04,
        8.15724667e-04, 1.27127460e-03, 6.72946513e-03, 2.13394182e-02,
        9.58066857e-04, 5.27181044e-04, 1.06384814e-03, 2.887

In [80]:
grid_search_svm.best_estimator_

SVC(C=0.5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [81]:
grid_search_svm.best_index_

1

In [83]:
(0.69+0.604+0.640+0.635+0.654)/5

0.6446